In [2]:
import torch
import os
from IPython.display import Image, clear_output  # to display images
from hyperopt import fmin, tpe, hp, Trials
from ultralytics import YOLO

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 2.0.1+cu117 (NVIDIA GeForce GTX 1660 Ti)


In [4]:
def objective(params):
    # Define el modelo con los parámetros dados
    model = YOLO('yolov8n-cls.pt')
    
    # Entrena el modelo
    model.train(data="/tf/data/Mammographies/yolo_data/abnormality_classification",
                task="classify",
                epochs=10,
                dropout=params["dropout"],
                lr0=params["lr"],                
                batch=2,
                augment=True,
                imgsz=320)
    
    # Evalúa el modelo
    metrics = model.val()
    
    # Devuelve la métrica a optimizar
    return -metrics.results_dict["metrics/accuracy_top1"]

# Define los rangos de los hiperparámetros
space = {
    'dropout': hp.choice('dropout',[0.0, 0.2, 0.4]),
    'lr': hp.loguniform('lr', -6, -1),
}

trials = Trials()

# Realiza la búsqueda de hiperparámetros
best = fmin(objective, space, algo=tpe.suggest, max_evals=15, trials=trials)

  0%|          | 0/15 [00:00<?, ?trial/s, best loss=?]

Transferred 150/355 items from pretrained weights
New https://pypi.org/project/ultralytics/8.0.109 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.98 🚀 Python-3.8.10 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce GTX 1660 Ti, 5945MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/tf/data/Mammographies/yolo_data/classification, epochs=1, patience=50, batch=2, imgsz=300, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize

  0%|          | 0/15 [00:00<?, ?trial/s, best loss=?]


RuntimeError: Dataset '/tf/data/Mammographies/yolo_data/classification' error ❌ [Errno 21] Is a directory: '/tf/data/Mammographies/yolo_data/classification'

In [ ]:
trials.trials

In [ ]:
best = space_eval(space, best)

In [5]:
# Define el modelo con los parámetros dados
model = YOLO('yolov8n-cls.pt')

# Entrena el modelo
model.train(data="/tf/data/Mammographies/yolo_data/abnormality_classification",
            epochs=100,
            dropout=best["dropout"],
            lr0=best["lr"],                
            batch=2,
            augment=True,
            imgsz=320)

Ultralytics YOLOv8.0.109 🚀 Python-3.8.10 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce GTX 1660 Ti, 5945MiB)
yolo/engine/trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=/tf/data/Mammographies/yolo_data/abnormality_classification, epochs=1, patience=50, batch=2, imgsz=300, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.2, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=Fal

In [6]:
# Evalúa el modelo
metrics = model.val()

Ultralytics YOLOv8.0.109 🚀 Python-3.8.10 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce GTX 1660 Ti, 5945MiB)
YOLOv8n-cls summary (fused): 73 layers, 1443847 parameters, 0 gradients
               classes   top1_acc   top5_acc: 100%|██████████| 1113/1113 [00:04<00:00, 257.11it/s]
                   all      0.598      0.996
Speed: 0.1ms preprocess, 1.6ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/val


In [7]:
metrics

ultralytics.yolo.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.yolo.utils.metrics.ConfusionMatrix object at 0x7ff582b0d940>
fitness: 0.9955056309700012
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.5977528095245361, 'metrics/accuracy_top5': 0.9955056309700012, 'fitness': 0.9955056309700012}
speed: {'preprocess': 0.1463603973388672, 'inference': 1.584939742356204, 'loss': 0.006503844529055477, 'postprocess': 0.006183345666092434}
top1: 0.5977528095245361
top5: 0.9955056309700012

In [8]:
model.export()

Ultralytics YOLOv8.0.109 🚀 Python-3.8.10 torch-2.0.1+cu117 CPU

PyTorch: starting from runs/classify/train3/weights/best.pt with input shape (2, 3, 320, 320) BCHW and output shape(s) (2, 7) (2.8 MB)

TorchScript: starting export with torch 2.0.1+cu117...
TorchScript: export success ✅ 0.6s, saved as runs/classify/train3/weights/best.torchscript (5.6 MB)

Export complete (0.7s)
Results saved to /tf/code/YOLOv8/abnormalities_type_classificator/runs/classify/train3/weights
Predict:         yolo predict task=classify model=runs/classify/train3/weights/best.torchscript imgsz=320 
Validate:        yolo val task=classify model=runs/classify/train3/weights/best.torchscript imgsz=320 data=/tf/data/Mammographies/yolo_data/abnormality_classification 
Visualize:       https://netron.app


'runs/classify/train3/weights/best.torchscript'

In [15]:
from PIL import Image
model_load=YOLO("./runs/classify/train3/weights/best.pt")
image=Image.open(f"/tf/data/Mammographies/yolo_data/abnormality_classification/train/Distortion/2.png").convert("RGB")
results=model_load(image)[0] #Saved on runs/detect/predict
print(results.probs.data.tolist())
results.names


0: 320x320 Calcification 0.57, Mass 0.40, Cluster 0.01, Assymetry 0.01, Distortion 0.01, 2.1ms
Speed: 1.2ms preprocess, 2.1ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 320)


[0.0075722625479102135, 0.5659735202789307, 0.0076529704965651035, 0.007340975105762482, 0.3993372321128845, 0.00685231015086174, 0.005270735826343298]


{0: 'Assymetry',
 1: 'Calcification',
 2: 'Cluster',
 3: 'Distortion',
 4: 'Mass',
 5: 'Spiculated Region',
 6: 'Unknown'}